In [10]:
import pandas as pd

df = pd.read_csv("faa_strikes.txt", sep=None, engine="python")
cols = {c.lower().strip(): c for c in df.columns}

date_col = next((cols[k] for k in cols if "date" in k), None)
if date_col is None:
    raise ValueError("Could not find a date column. Look for something like 'Flight Date' or 'Incident Date'.")

s = pd.to_datetime(df[date_col], errors="coerce", utc=False)

try:
    s = s.dt.tz_convert(None)
except Exception:
    s = s.dt.tz_localize(None)
df[date_col] = s

cutoff = pd.Timestamp("2013-12-31")
daily = (
    df.dropna(subset=[date_col])
      .loc[df[date_col] <= cutoff]
      .assign(date=lambda x: x[date_col].dt.normalize())
      .groupby("date", as_index=False)
      .size()
      .rename(columns={"size": "daily_strikes"})
      .sort_values("date")
      .reset_index(drop=True)
)

daily["running_total"] = daily["daily_strikes"].cumsum()

running_total_strikes = daily[["date", "daily_strikes", "running_total"]]
running_total_strikes.head(), running_total_strikes.tail()

(        date  daily_strikes  running_total
 0 2000-01-02              1              1
 1 2000-01-03              2              3
 2 2000-01-05              3              6
 3 2000-01-06              1              7
 4 2000-01-08              4             11,
            date  daily_strikes  running_total
 4770 2013-12-26              3          24090
 4771 2013-12-27              6          24096
 4772 2013-12-28              1          24097
 4773 2013-12-29              2          24099
 4774 2013-12-30              4          24103)

In [11]:
import pandas as pd

df = pd.read_csv("faa_strikes.txt", sep=None, engine="python")
cols = {c.lower().strip(): c for c in df.columns}

state_col = None
for k in cols:
    if k == "state" or k.endswith(": state") or "state" in k:
        state_col = cols[k]
        break

cost_col = None
for k in cols:
    if "cost" in k and ("total" in k or "$" in k or "usd" in k or "cost" == k):
        cost_col = cols[k]
        break

if state_col is None or cost_col is None:
    raise ValueError("Could not find state and/or total cost column. Look for columns like 'State' and 'Cost: Total $'.")

cost_series = pd.to_numeric(df[cost_col].astype(str).str.replace(r"[^0-9\.\-]", "", regex=True), errors="coerce")

agg = (
    df.assign(_cost=cost_series)
      .groupby(state_col, as_index=False)["_cost"].sum()
      .rename(columns={"_cost": "total_cost"})
)

agg["ranking"] = agg["total_cost"].rank(method="dense", ascending=False)
damage_state = agg.loc[agg["ranking"] == 3].sort_values(state_col).reset_index(drop=True)

damage_state

,Origin State,total_cost,ranking
0,California,29671432,3.0


In [18]:
import pandas as pd

df = pd.read_csv("faa_strikes.txt", sep=None, engine="python")
cols = {c.lower().strip(): c for c in df.columns}

atype_col = None
for k in cols:
    if "aircraft" in k and "type" in k:
        atype_col = cols[k]
        break

sg_col = None
for k in cols:
    if "species" in k and "group" in k:
        sg_col = cols[k]
        break

cost_col = None
for k in cols:
    if "cost" in k and ("total" in k or "$" in k or "usd" in k or "cost" == k):
        cost_col = cols[k]
        break

if atype_col is None or sg_col is None or cost_col is None:
    raise ValueError("Missing a required column: 'Aircraft: Type', 'Species: Group', or total cost column.")

cost = pd.to_numeric(df[cost_col].astype(str).str.replace(r"[^0-9\.\-]", "", regex=True), errors="coerce").fillna(0)

g = (
    df.assign(_cost=cost)
      .groupby([atype_col, sg_col], as_index=False)["_cost"].sum()
      .rename(columns={"_cost": "total_cost"})
)

g["rank"] = g.groupby(atype_col)["total_cost"].rank(method="dense", ascending=False)

second_costliest_species = (
    g.loc[g["rank"] == 2]
     .sort_values([atype_col, "total_cost"], ascending=[True, False])
     .reset_index(drop=True)
)

second_costliest_species

,Aircraft: Type,Wildlife: Species Group,total_cost,rank
0,Airplane,"Kites, Hawks, Eagles",48989287,2.0
1,Helicopter,Vultures,116748,2.0


In [17]:
import pandas as pd

df = pd.read_csv("faa_strikes.txt", sep=None, engine="python")
cols = {c.lower().strip(): c for c in df.columns}

date_col = None
for k in cols:
    if "date" in k:
        date_col = cols[k]
        break
if date_col is None:
    raise ValueError("Could not find a date column.")

df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
daily = (
    df.dropna(subset=[date_col])
      .assign(date=lambda x: x[date_col].dt.normalize())
      .groupby("date", as_index=False)
      .size()
      .rename(columns={"size": "daily_strikes"})
      .sort_values("date")
      .reset_index(drop=True)
)

daily["previous_day"] = daily["daily_strikes"].shift(1)
daily["delta_strikes"] = daily["daily_strikes"] - daily["previous_day"]

greatest_strike_increase = daily.sort_values("delta_strikes", ascending=False).reset_index(drop=True)

greatest_strike_increase.head(10)

,date,daily_strikes,previous_day,delta_strikes
0,2010-10-29 00:00:00+00:00,25,7.0,18.0
1,2014-08-25 00:00:00+00:00,25,8.0,17.0
2,2012-06-28 00:00:00+00:00,21,5.0,16.0
3,2009-07-07 00:00:00+00:00,21,7.0,14.0
4,2013-11-04 00:00:00+00:00,22,8.0,14.0
5,2014-10-28 00:00:00+00:00,24,10.0,14.0
6,2009-10-05 00:00:00+00:00,18,5.0,13.0
7,2009-09-13 00:00:00+00:00,16,3.0,13.0
8,2012-08-13 00:00:00+00:00,23,10.0,13.0
9,2013-08-19 00:00:00+00:00,28,15.0,13.0


In [16]:
import pandas as pd

df = pd.read_csv("faa_strikes.txt", sep=None, engine="python")
cols = {c.lower().strip(): c for c in df.columns}

date_col = None
for k in cols:
    if "date" in k:
        date_col = cols[k]
        break

atype_col = None
for k in cols:
    if "aircraft" in k and "type" in k:
        atype_col = cols[k]
        break

if date_col is None or atype_col is None:
    raise ValueError("Missing required columns: need a date column and 'Aircraft: Type' (or similar).")

df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
daily_type = (
    df.dropna(subset=[date_col, atype_col])
      .assign(date=lambda x: x[date_col].dt.normalize())
      .groupby([atype_col, "date"], as_index=False)
      .size()
      .rename(columns={"size": "daily_strikes"})
      .sort_values([atype_col, "date"])
      .reset_index(drop=True)
)

daily_type["previous_day"] = daily_type.groupby(atype_col)["daily_strikes"].shift(1)
daily_type["delta_strikes"] = daily_type["daily_strikes"] - daily_type["previous_day"]

daily_type["max_rank"] = daily_type.groupby(atype_col)["delta_strikes"].rank(method="first", ascending=False)

greatest_strike_increase_type = (
    daily_type.loc[daily_type["max_rank"] == 1]
              .drop(columns=["max_rank"])
              .sort_values([atype_col, "delta_strikes"], ascending=[True, False])
              .reset_index(drop=True)
)

greatest_strike_increase_type

,Aircraft: Type,date,daily_strikes,previous_day,delta_strikes
0,Airplane,2014-08-25 00:00:00+00:00,25,8.0,17.0
1,Helicopter,2010-07-21 00:00:00+00:00,2,1.0,1.0
